<a href="https://colab.research.google.com/github/yalopez84/Interpretable_method/blob/master/Interpretable_Triple_Prediction_Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install sentence_transformers
!pip install umap-learn 
!pip install hdbscan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 84 kB 1.9 MB/s 
     |████████████████████████████████| 4.4 MB 15.5 MB/s 
     |████████████████████████████████| 1.2 MB 58.1 MB/s 
     |████████████████████████████████| 101 kB 10.2 MB/s 
     |████████████████████████████████| 596 kB 58.2 MB/s 
     |████████████████████████████████| 6.6 MB 38.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.1-py3-none-any.whl size=125774 sha256=5bf1b30a1c56950a1c3c34a718ae56bd8987f7d0314af99a3b0c2b7c8a94a257
  Stored in directory: /root/.cache/pip/wheels/58/27/2f/708b4f002c226e57b6243769da345c650633175c7634f93365
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-

In [5]:
from sentence_transformers import SentenceTransformer
import os
import csv
import sys
import random
from tqdm import tqdm, trange
import logging
import pdb
import umap
import hdbscan
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbs
from torch.utils.tensorboard import SummaryWriter
writter = SummaryWriter(log_dir='tensorboard_embedding_custom2')
logger = logging.getLogger(__name__)

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
pathFile="/content/drive/MyDrive/Almacen_phd/FB13/"
modelName="/content/drive/MyDrive/Almacen_phd/FB13/sbert_test_mnr2"
embeddings_large_file="/content/drive/MyDrive/Almacen_phd/FB13/embedding_large.pt"
umap_embeddings_file="/content/drive/MyDrive/Almacen_phd/FB13/umap_embeddings.pt"
asignacion_clusters="/content/drive/MyDrive/Almacen_phd/FB13/clusters_labels"
os.chdir(pathFile)


# Preprocessing-Start

In [8]:
class InputExample(object):

    def __init__(self, guid, text_a, text_b=None, text_c=None, label=None):
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.text_c = text_c
        self.label = label

In [9]:
class Triple(object):
    def __init__(self, guid, subject , predicate , obj, label):
        self.guid=guid
        self.subject=subject
        self.predicate=predicate
        self.obj=obj
        self.label=label

In [10]:
class ExampleTruncado(object):

    def __init__(self, text, label=None):
        self.text = text
        self.label = label

In [11]:
class DataProcessor(object):
  
    def get_train_examples(self, data_dir):
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        raise NotImplementedError()

    def get_labels(self, data_dir):
        raise NotImplementedError()

    def get_train_with_negatives(self,data_dir):
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        with open(input_file, "r", encoding="utf-8") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []            
            for line in reader:
                lines.append(line)
            return lines
            
    @classmethod
    def _read_train_with_negatives(cls, input_file, quotechar=None):
        with open(input_file, "r", encoding="utf-8") as f:
            reader = csv.reader(f, delimiter=" ", quotechar=quotechar)
            lines = []
            for line in reader:         
                lines.append(line)
            return lines

In [12]:
class KGProcessor(DataProcessor):
    def __init__(self):
        self.labels = set()
    
    def get_train_examples(self, data_dir):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train", data_dir)
        
    def get_dev_examples(self, data_dir):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev", data_dir)

    def get_test_examples(self, data_dir):
        return self._create_examples(
          self._read_tsv(os.path.join(data_dir, "test_reduced.tsv")), "test", data_dir)
        
    def get_train_with_negatives(self,data_dir):
        return self._create_examples_with_negatives(
          self._read_train_with_negatives(os.path.join(data_dir, "train_reduced_with_negatives.txt")), "embeddings", data_dir)

    def get_relations(self, data_dir):
        with open(os.path.join(data_dir, "relations.txt"), 'r') as f:
            lines = f.readlines()
            relations = []
            for line in lines:
                relations.append(line.strip())
        return relations

    def get_labels(self, data_dir):
        return ["0", "1"]

    def get_entities(self, data_dir):
        with open(os.path.join(data_dir, "entities.txt"), 'r') as f:
            lines = f.readlines()
            entities = []
            for line in lines:
                entities.append(line.strip())
        return entities

    def get_prediction_test_triples(self, data_dir):
        with open(os.path.join(data_dir, "test_results.txt"), 'r') as f:
            lines = f.readlines()
            predictions = []
            for line in lines:               
                predictions.append(line.split()[2])
        return predictions

    def get_train_triples(self, data_dir):
        return self._read_train_with_negatives(os.path.join(data_dir, "train_reduced_with_negatives.txt"))    

    def get_dev_triples(self, data_dir):
        return self._read_tsv(os.path.join(data_dir, "dev.tsv"))

    def get_test_triples(self, data_dir):
        return self._read_tsv(os.path.join(data_dir, "test_reduced.tsv"))

    def _create_examples(self, lines, set_type, data_dir):
        ent2text = {}
        with open(os.path.join(data_dir, "entity2text.txt"), 'r') as f:
            ent_lines = f.readlines()
            for line in tqdm(ent_lines):
                temp = line.strip().split('\t')
                if len(temp) == 2:
                    ent2text[temp[0]] = temp[1]

        entities = list(ent2text.keys())

        rel2text = {}
        with open(os.path.join(data_dir, "relation2text.txt"), 'r') as f:
            rel_lines = f.readlines()
            for line in rel_lines:
                temp = line.strip().split('\t')
                rel2text[temp[0]] = temp[1]      

        lines_str_set = set(['\t'.join(line) for line in lines])
        examples = []
        for (i, line) in enumerate(lines):
            auxiliar=["loke_joseph_of_austria_palatine_of_hungary","united_srena_gale", "archduke_joseph_of_austria_palatine_of_hungary"]
            flag_aux=False
            if line[0] or line[2] not in auxiliar:
                head_ent_text = ent2text[line[0]]
                tail_ent_text = ent2text[line[2]]
                relation_text = rel2text[line[1]]
                flag_aux=True
            else:
                head_ent_text = line[0]
                tail_ent_text = line[2]
                relation_text = line[1]
                flag_aux=True

            if set_type == "dev" or set_type == "test" and flag_aux:
                triple_label = line[3]
                if triple_label == "1":
                    label = "1"
                else:
                    label = "0"

                guid = "%s-%s" % (set_type, i)
                text_a = head_ent_text
                text_b = relation_text
                text_c = tail_ent_text 
                self.labels.add(label)
                examples.append(
                    InputExample(guid=guid, text_a=text_a, text_b=text_b, text_c = text_c, label=label))
                
            elif set_type == "train" and flag_aux:
                guid = "%s-%s" % (set_type, i)
                text_a = head_ent_text
                text_b = relation_text
                text_c = tail_ent_text 
                examples.append(
                    InputExample(guid=guid, text_a=text_a, text_b=text_b, text_c = text_c, label="1"))

                rnd = random.random()
                guid = "%s-%s" % (set_type + "_corrupt", i)
                if rnd <= 0.5:
                    # corrupting head
                    tmp_head = ''
                    while True:
                        tmp_ent_list = set(entities)
                        tmp_ent_list.remove(line[0])
                        tmp_ent_list = list(tmp_ent_list)
                        tmp_head = random.choice(tmp_ent_list)
                        tmp_triple_str = tmp_head + '\t' + line[1] + '\t' + line[2]
                        if tmp_triple_str not in lines_str_set:
                            break                    
                    tmp_head_text = ent2text[tmp_head]
                    examples.append(
                        InputExample(guid=guid, text_a=tmp_head_text, text_b=text_b, text_c = text_c, label="0"))       
                else:
                    # corrupting tail
                    tmp_tail = ''
                    while True:
                        tmp_ent_list = set(entities)
                        tmp_ent_list.remove(line[2])
                        tmp_ent_list = list(tmp_ent_list)
                        tmp_tail = random.choice(tmp_ent_list)
                        tmp_triple_str = line[0] + '\t' + line[1] + '\t' + tmp_tail
                        if tmp_triple_str not in lines_str_set:
                            break
                    tmp_tail_text = ent2text[tmp_tail]
                    examples.append(
                        InputExample(guid=guid, text_a=text_a, text_b=text_b, text_c = tmp_tail_text, label="0"))      
                                                            
        return examples

    def _create_examples_with_negatives(self, lines, set_type, data_dir):
        
        ent2text = {}
        with open(os.path.join(data_dir, "entity2text.txt"), 'r') as f:
            ent_lines = f.readlines()
            for line in tqdm(ent_lines):
                temp = line.strip().split('\t')
                if len(temp) == 2:
                    ent2text[temp[0]] = temp[1]

        entities = list(ent2text.keys())

        rel2text = {}
        with open(os.path.join(data_dir, "relation2text.txt"), 'r') as f:
            rel_lines = f.readlines()
            for line in rel_lines:
                temp = line.strip().split('\t')
                rel2text[temp[0]] = temp[1]      

        examples = []
        for (i, line) in enumerate(lines):
            auxiliar=["loke_joseph_of_austria_palatine_of_hungary","united_srena_gale", "archduke_joseph_of_austria_palatine_of_hungary"]
            flag_aux=False
            if len(line)==5:
                if line[1] or line[3] not in auxiliar:               
                    guid=line[0]
                    head_ent_text = ent2text[line[1]]                
                    relation_text = rel2text[line[2]]
                    tail_ent_text = ent2text[line[3]]
                    triple_label = line[4]
                    examples.append(
                        InputExample(guid=guid, text_a=head_ent_text, text_b=relation_text, text_c = tail_ent_text, label=triple_label))     
                                                           
        return examples

In [13]:
def convert_examples_to_examplesTruncados(examples, max_seq_length, label_list, tokenizer=None, print_info = True):
    label_map = {label : i for i, label in enumerate(label_list)}
    examples_truncados = []
    logger.info("")
    logger.info("Writing examples truncados")
    logger.info("max_seq_length %d" % (max_seq_length))
    separador="////////////// "

    for (ex_index, example) in tqdm(enumerate(examples)):
        
        tokens_a = example.text_a.split()
        tokens_b = None
        tokens_c = None
       
        if example.text_b and example.text_c:
            tokens_b = example.text_b.split()
            tokens_c = example.text_c.split()
           
            _truncate_seq_triple(tokens_a, tokens_b, tokens_c, max_seq_length)
        else:
            if len(tokens_a) > max_seq_length:
                tokens_a = tokens_a[:(max_seq_length)]
        tokens =tokens_a.copy()

        if tokens_b:
            tokens.extend(tokens_b)           
        if tokens_c:
            tokens.extend(tokens_c)        

        padding = ["0"] * (max_seq_length - len(tokens))
        tokens.extend(padding)
        assert len(tokens) == max_seq_length

        examples_truncados.append(
                ExampleTruncado(text=tokens,label=example.label))
    return examples_truncados

In [14]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    while True:
        sujeto=tokens_a.split()
        relation=tokens_b.split()
        total_length = len(sujeto) + len(relation)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [15]:
def _truncate_seq_triple(tokens_a, tokens_b, tokens_c, max_length):
    while True:
        total_length = len(tokens_a) + len(tokens_b) + len(tokens_c)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b) and len(tokens_a) > len(tokens_c):
            tokens_a.pop()
        elif len(tokens_b) > len(tokens_a) and len(tokens_b) > len(tokens_c):
            tokens_b.pop()
        elif len(tokens_c) > len(tokens_a) and len(tokens_c) > len(tokens_b):
            tokens_c.pop()
        else:
            tokens_c.pop()

# Preprocessing-End

In [16]:
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt = '%m/%d/%Y %H:%M:%S',
    level = logging.INFO)

processor = KGProcessor()  
label_list = processor.get_labels(pathFile)

In [17]:
#Test triples
ids_list=[]
subjets_list=[]
predicates_list=[]
objects_list=[]
labels_list=[]
_examples_test = processor.get_test_examples(pathFile) #se cargan las primeras 100 tripletas del conjunto test.tsv
for sample in _examples_test:
    ids_list.append(sample.guid)
    subjets_list.append(sample.text_a)
    predicates_list.append(sample.text_b)
    objects_list.append(sample.text_c)
    labels_list.append(sample.label)

data_ids=pd.Series(ids_list)
data_subjects=pd.Series(subjets_list)
data_predicates=pd.Series(predicates_list)
data_objects=pd.Series(objects_list)
data_labels=pd.Series(labels_list)

frame_triples_test=pd.DataFrame({'Id_Triple_Test':data_ids,'Subjects':data_subjects,'Predicates':data_predicates, "Objects":data_objects, "Label":data_labels})
kg_Bert_predictions_list=processor.get_prediction_test_triples(os.path.join(pathFile,"output_FB13" ))
frame_triples_test["KG BERT Prediction"]=kg_Bert_predictions_list
frame_triples_test

100%|██████████| 75042/75042 [00:00<00:00, 233731.53it/s]


,Id_Triple_Test,Subjects,Predicates,Objects,Label,KG BERT Prediction
0,test-0,Umberto I (Italian: Umberto Rainerio Carlo Ema...,cause of death,Tyrannicide is the killing or assassination of...,1,1
1,test-1,"John Glenn Beall Jr. (June 19, 1927 – March 24...",nationality,"The United States of America (USA), commonly k...",1,1
2,test-2,John Atkinson Grimshaw (6 September 1836 – 13 ...,gender,A male (♂) organism is the physiological sex t...,1,1
3,test-3,"Henry John Heinz (October 11, 1844 – May 14, 1...",nationality,"Germany (German: Deutschland, German pronuncia...",1,1
4,test-4,"Ernest Lundeen (August 4, 1878 – August 31, 19...",profession,A politician is a person active in party polit...,1,1
...,...,...,...,...,...,...
2248,test-2248,"Lew Cody (born Louis Joseph Côté; February 22,...",gender,A male (♂) organism is the physiological sex t...,0,1
2249,test-2249,Leo Szilard (; Hungarian: Szilárd Leó [ˈsilaːr...,profession,A laborer or labourer is a person who works in...,0,0
2250,test-2250,Quentin Crisp (born Denis Charles Pratt; (1908...,profession,"A disc jockey, more commonly abbreviated as DJ...",0,0
2251,test-2251,Tobias Mayer (17 February 1723 – 20 February 1...,nationality,"East Germany, officially the German Democratic...",0,1


In [18]:
#Train triples
ids_list2=[]
subjets_list2=[]
predicates_list2=[]
objects_list2=[]
labels_list2=[]
_examples_train=processor.get_train_with_negatives(pathFile) 
for sample2 in _examples_train:
    ids_list2.append(sample2.guid)
    subjets_list2.append(sample2.text_a)
    predicates_list2.append(sample2.text_b)
    objects_list2.append(sample2.text_c)
    labels_list2.append(sample2.label)

data2_ids=pd.Series(ids_list2)
data2_subjects=pd.Series(subjets_list2)
data2_predicates=pd.Series(predicates_list2)
data2_objects=pd.Series(objects_list2)
data2_labels=pd.Series(labels_list2)

frame_triples_train=pd.DataFrame({'Id_Triple_Training':data2_ids, 'Subjects':data2_subjects,'Predicates':data2_predicates, "Objects":data2_objects, "Label":data2_labels})
frame_triples_train

100%|██████████| 75042/75042 [00:00<00:00, 275090.77it/s]


,Id_Triple_Training,Subjects,Predicates,Objects,Label
0,train-0,"Denys Arthur Rayner DSC & Bar, VRD, RNVR (9 Fe...",cause of death,Cancer is a group of diseases involving abnorm...,1
1,train_corrupt-0,"Denys Arthur Rayner DSC & Bar, VRD, RNVR (9 Fe...",cause of death,Thomas Langlois Lefroy (8 January 1776 – 4 May...,0
2,train-1,Karl Wilhelm Otto Lilienthal (23 May 1848 – 10...,cause of death,"In aviation, an accident is defined by the Con...",1
3,train_corrupt-1,Karl Wilhelm Otto Lilienthal (23 May 1848 – 10...,cause of death,"John Manners, 2nd Duke of Rutland KG (18 Septe...",0
4,train-2,Rajiv Ratna Gandhi ( (listen); 20 August 1944 ...,cause of death,Assassination is the act of killing a prominen...,1
...,...,...,...,...,...
22159,train_corrupt-11079,"Irkutsk (Russian: Иркутск, IPA: [ɪrˈkutsk]; Bu...",profession,"Engineers, as practitioners of engineering, ar...",0
22160,train-11080,Radhanath Sikdar (Bengali: রাধানাথ শিকদার; Oct...,profession,A mathematician is someone who uses an extensi...,1
22161,train_corrupt-11080,"Archduke Karl Salvator of Austria, in Italian ...",profession,A mathematician is someone who uses an extensi...,0
22162,train-11081,Peter Joseph William Debye (; Dutch: [dəˈbɛiə...,profession,A scientist is someone who conducts scientific...,1


In [19]:
id_input_triple=input("Enter the test triple id to be explained:")
frame_triples_test[frame_triples_test["Id_Triple_Test"]==id_input_triple]

Enter the test triple id to be explained:test-2250


,Id_Triple_Test,Subjects,Predicates,Objects,Label,KG BERT Prediction
2250,test-2250,Quentin Crisp (born Denis Charles Pratt; (1908...,profession,"A disc jockey, more commonly abbreviated as DJ...",0,0


In [20]:
#Turn training triples + target test triple into sequences 
_examples_train_truncates= convert_examples_to_examplesTruncados(_examples_train, 512,label_list)
X_train=[]
X_train2=[]
for sample in _examples_train_truncates:
    X_train.append(" ".join(sample.text))

target_triple_test=list(e for e in _examples_test if e.guid  == id_input_triple)
target_triple_sequence_truncate=convert_examples_to_examplesTruncados(target_triple_test, 512,label_list)
X_train2.append(" ".join(target_triple_sequence_truncate[0].text))

06/23/2022 13:56:05 - INFO - __main__ -   
06/23/2022 13:56:05 - INFO - __main__ -   Writing examples truncados
06/23/2022 13:56:05 - INFO - __main__ -   max_seq_length 512
22164it [00:03, 7241.87it/s]
06/23/2022 13:56:09 - INFO - __main__ -   
06/23/2022 13:56:09 - INFO - __main__ -   Writing examples truncados
06/23/2022 13:56:09 - INFO - __main__ -   max_seq_length 512
1it [00:00, 1222.12it/s]


In [42]:
model = SentenceTransformer(modelName)
if os.path.exists(embeddings_large_file):
    embeddings_large=torch.load(embeddings_large_file)
else:
    embeddings_large = model.encode(X_train, show_progress_bar=True) 
    torch.save(embeddings_large,embeddings_large_file) 

embeddings_large_target_triple=model.encode(X_train2, show_progress_bar=True) 
embeddings_large=np.append(embeddings_large,embeddings_large_target_triple, axis=0)

06/23/2022 14:31:13 - INFO - sentence_transformers.SentenceTransformer -   Load pretrained SentenceTransformer: /content/drive/MyDrive/Almacen_phd/FB13/sbert_test_mnr2
06/23/2022 14:31:21 - INFO - sentence_transformers.SentenceTransformer -   Use pytorch device: cpu


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [44]:
#Dimension reduction with UMAP
umap_embeddings = umap.UMAP(n_neighbors=15, 
                            n_components=5,
                            min_dist=0.0,
                            metric='cosine').fit_transform(embeddings_large)
torch.save(umap_embeddings, umap_embeddings_file)
pd.DataFrame(umap_embeddings)

,0,1,2,3,4
0,7.699249,10.836530,7.818552,-1.165031,-4.591991
1,11.104531,6.627504,10.673512,5.016499,-1.764336
2,9.349195,6.838246,6.429531,5.110744,-1.565489
3,10.988402,6.155725,10.782904,5.589000,-2.038962
4,8.788750,9.962027,8.279836,2.589678,0.693110
...,...,...,...,...,...
22160,11.021442,5.716213,11.503374,4.021300,-2.047497
22161,11.013853,5.748580,11.479890,4.109618,-2.054622
22162,10.847095,6.385328,10.949208,3.610063,-1.547866
22163,10.844790,6.378306,10.949274,3.592330,-1.546531


In [45]:
#To cluster embeddings with HDBSCAN
cluster = hdbscan.HDBSCAN(min_cluster_size=5, 
                        metric='euclidean', 
                        cluster_selection_method='eom'
                        ).fit(umap_embeddings)

#Saving each triple with its cluster label
with open(asignacion_clusters, "w") as writer:
    writer.write("Clusters asignados\n")
    writer.write("id de tripleta -------cluster\n")
    for id, idcluster in enumerate(cluster.labels_):
        writer.write("%d ---------- %d\n" % (id, idcluster))    

print(f"Number of clusters = {np.unique(cluster.labels_).shape[0] - 1}") 

Number of clusters = 635


In [46]:
#Getting the positions of triples in the same cluster of the target triple
cluster_target=cluster.labels_[len(cluster.labels_)-1]
position_triples_targets=[pos for pos,value in enumerate(cluster.labels_) if value==cluster_target]

In [50]:
#Target triple set
target_triples_set = [example for pos, example in enumerate(_examples_train) if pos in position_triples_targets]
ids_list3=[]
subjets_list3=[]
predicates_list3=[]
objects_list3=[]
labels_list3=[]
for sample3 in target_triples_set:
    ids_list3.append(sample3.guid)
    subjets_list3.append(sample3.text_a)
    predicates_list3.append(sample3.text_b)
    objects_list3.append(sample3.text_c)
    labels_list3.append(sample3.label)

data3_ids=pd.Series(ids_list3)
data3_subjects=pd.Series(subjets_list3)
data3_predicates=pd.Series(predicates_list3)
data3_objects=pd.Series(objects_list3)
data3_labels=pd.Series(labels_list3)
frame3_target_triple_train=pd.DataFrame({'Id_triple_Training':data3_ids, 'Subjects':data3_subjects,'Predicates':data3_predicates, "Objects":data3_objects, "Label":data3_labels})
frame3_target_triple_train

,Id_triple_Training,Subjects,Predicates,Objects,Label
0,train_corrupt-0,"Denys Arthur Rayner DSC & Bar, VRD, RNVR (9 Fe...",cause of death,Thomas Langlois Lefroy (8 January 1776 – 4 May...,0
1,train_corrupt-1,Karl Wilhelm Otto Lilienthal (23 May 1848 – 10...,cause of death,"John Manners, 2nd Duke of Rutland KG (18 Septe...",0
2,train_corrupt-5,Murder in Small Town X (sometimes abbreviated ...,cause of death,Sanjay Gandhi (14 December 1946 – 23 June 1980...,0
3,train_corrupt-6,"William Cowper Brann (January 4, 1855 – April ...",cause of death,James Francis Edward Keith (11 June 1696 – 14 ...,0
4,train_corrupt-9,"Warren Mercer Oates (July 5, 1928 – April 3, 1...",cause of death,"Virginia Valli (January 18, 1895 – September 2...",0
...,...,...,...,...,...
6367,train_corrupt-11074,"Franklin Swift Billings Jr. (June 5, 1922 – Ma...",profession,"The Holy Roman Emperor, officially the Emperor...",0
6368,train-11075,Baron Stow (1801–1869) was a Boston Baptist mi...,profession,A writer is a person who uses written words in...,1
6369,train_corrupt-11075,Baron Stow (1801–1869) was a Boston Baptist mi...,profession,"Martin John Spalding (May 23, 1810 – February ...",0
6370,train-11076,"László Tisza (July 7, 1907 – April 15, 2009) w...",nationality,"The United States of America (USA), commonly k...",1


In [51]:
#Target triple set embeddings
umap_embeddings_target_triple_set= [vector for pos, vector in enumerate(umap_embeddings[:-1]) if pos in position_triples_targets]
umap_embeddings_target_triple_set_df=pd.DataFrame(umap_embeddings_target_triple_set)

ids_triple_target_set= [sample.guid for pos, sample in enumerate(_examples_train) if pos in position_triples_targets]
labels_triple_target_set= [sample.label for pos, sample in enumerate(_examples_train) if pos in position_triples_targets]

umap_embeddings_target_triple_set_df["Id_triple_Training"]=ids_triple_target_set
umap_embeddings_target_triple_set_df["Label"]=labels_triple_target_set

umap_embeddings_target_triple_set_df.to_excel("Embeddings_Regresion.xlsx")
umap_embeddings_target_triple_set_df

,0,1,2,3,4,Id_triple_Training,Label
0,11.104531,6.627504,10.673512,5.016499,-1.764336,train_corrupt-0,0
1,10.988402,6.155725,10.782904,5.589000,-2.038962,train_corrupt-1,0
2,10.915964,6.262773,10.499018,5.531379,-2.002148,train_corrupt-5,0
3,10.943674,6.533581,10.651723,5.417235,-1.547288,train_corrupt-6,0
4,10.080222,7.322797,10.902537,5.328524,-2.955758,train_corrupt-9,0
...,...,...,...,...,...,...,...
6367,8.610104,9.895626,10.154102,2.333625,1.506434,train_corrupt-11074,0
6368,11.339172,12.419613,11.390947,0.703270,1.131583,train-11075,1
6369,11.005245,6.039375,11.047254,5.031971,-2.190608,train_corrupt-11075,0
6370,10.535950,10.427906,9.248152,-4.343508,4.182976,train-11076,1


In [52]:
#Logistic Regression Prediction
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(umap_embeddings_target_triple_set_df.iloc[:,:5],umap_embeddings_target_triple_set_df.iloc[:,6]) 

pred = lr.predict(umap_embeddings[-1,:].reshape(1,5))
pred
log_regresion=pred[0]
log_regresion

'0'

In [53]:
#Decision Tree Prediction
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

dt = DecisionTreeClassifier(max_depth=10)
dt.fit(umap_embeddings_target_triple_set_df.iloc[:,:5],umap_embeddings_target_triple_set_df.iloc[:,6])
pred = dt.predict(umap_embeddings[-1,:].reshape(1,5))
arbol_decision=pred[0]
arbol_decision

'0'

In [54]:
#Representar arbol en texto plano
from sklearn.tree import export_text
r = export_text(dt)
print(r)

|--- feature_1 <= 7.50
|   |--- feature_4 <= -1.43
|   |   |--- feature_4 <= -3.03
|   |   |   |--- feature_1 <= 6.79
|   |   |   |   |--- feature_3 <= 5.42
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- feature_3 >  5.42
|   |   |   |   |   |--- feature_1 <= 6.14
|   |   |   |   |   |   |--- feature_0 <= 10.99
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- feature_0 >  10.99
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- feature_1 >  6.14
|   |   |   |   |   |   |--- class: 0
|   |   |   |--- feature_1 >  6.79
|   |   |   |   |--- feature_2 <= 12.19
|   |   |   |   |   |--- feature_2 <= 10.89
|   |   |   |   |   |   |--- feature_2 <= 10.37
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- feature_2 >  10.37
|   |   |   |   |   |   |   |--- feature_3 <= 5.42
|   |   |   |   |   |   |   |   |--- feature_0 <= 10.08
|   |   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |   |--- feature_0 >  10

In [55]:
from numpy.core.fromnumeric import squeeze
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
knc = KNeighborsClassifier(n_neighbors=5) 
rnc = RadiusNeighborsClassifier(radius=5) 
knc.fit(umap_embeddings_target_triple_set_df.iloc[:,:5],umap_embeddings_target_triple_set_df.iloc[:,6])
rnc.fit(umap_embeddings_target_triple_set_df.iloc[:,:5],umap_embeddings_target_triple_set_df.iloc[:,6])
knc_pred = knc.predict(umap_embeddings[-1,:].reshape(1,5))
knc.kneighbors(umap_embeddings[-1,:].reshape(1,5))
_, knc_neighbors = knc.kneighbors(umap_embeddings[-1,:].reshape(1,5))
knn_prediction=knc_pred[0]
vecinos=knc_neighbors.squeeze().tolist()
nearest_neighbors_df=umap_embeddings_target_triple_set_df.iloc[vecinos]
nearest_neighbors_df

,0,1,2,3,4,Id_triple_Training,Label
45,9.057560,10.443078,11.255684,2.755146,-1.641345,train_corrupt-98,0
1020,9.032115,10.418980,11.284353,2.720356,-1.607440,train_corrupt-1625,0
855,9.087613,10.491985,11.174622,2.808467,-1.665900,train-1369,1
2270,9.101866,10.460910,11.129881,2.749176,-1.605206,train_corrupt-3701,0
389,9.037788,10.453751,11.151381,2.693474,-1.550341,train_corrupt-601,0


In [56]:
from numpy.ma.core import append
#tripletas en version corta por los ids de los vecinos y tripleta target
id_tripletas=nearest_neighbors_df.iloc[:,5].unique()
#En este momento hay 200 tripletas en le fichero train_with_negatives.txt.
train_triples_sequences=processor.get_train_triples(pathFile)
triples=[]
triples_vecinos= [triple for triple in train_triples_sequences if triple[0] in id_tripletas]
for a in triples_vecinos:
    triples.append(Triple(guid=a[0],subject=a[1],predicate=a[2],obj=a[3], label=a[4]))

id=int(id_input_triple.split("-")[1])
#print(id)
tripletarget_sindescripcion=processor.get_test_triples(pathFile)[id]
print('\033[1m'+"Triple target \033[0m:  {}          {}         {}".format(tripletarget_sindescripcion[0], tripletarget_sindescripcion[1],tripletarget_sindescripcion[2]))
#print("Triple target: Subject %s   Predicate %s   Object %s " % tripletarget_sindescripcion[0], tripletarget_sindescripcion[1],tripletarget_sindescripcion[2])

ids_list4=[]
subjets_list4=[]
predicates_list4=[]
objects_list4=[]
labels_list4=[]

for sample4 in triples:
    ids_list4.append(sample4.guid)
    subjets_list4.append(sample4.subject)
    predicates_list4.append(sample4.predicate)
    objects_list4.append(sample4.obj)
    labels_list4.append(sample4.label)

data4_ids=pd.Series(ids_list4)
data4_subjects=pd.Series(subjets_list4)
data4_predicates=pd.Series(predicates_list4)
data4_objects=pd.Series(objects_list4)
data4_labels=pd.Series(labels_list4)

frame4_target_triple_train=pd.DataFrame({'Id_triple_Training':data4_ids, 'Subjects':data4_subjects,'Predicates':data4_predicates, "Objects":data4_objects, "Label":data4_labels})
frame4_target_triple_train


#****mostrar las componentes de la tripleta.


#Tripleta target y tripletas reducidas
#Tripletas target y ejemplos ampliados
#[[sample.text_a, sample2.text_b, sample2.text_c, sample.label] for sample in _examples_train if sample.guid in id_tripletas]
#[[sample.text_a, sample2.text_b, sample2.text_c, sample.label] for sample in _examples_train if sample.guid in id_tripletas]


Triple target :  quentin_crisp          profession         disc_jockey


,Id_triple_Training,Subjects,Predicates,Objects,Label
0,train_corrupt-98,disc_jockey,nationality,united_states,0
1,train_corrupt-601,terry_melcher,place_of_birth,ukraine,0
2,train-1369,keith_moon,nationality,sweden,1
3,train_corrupt-1625,johann_michael_heineccius,nationality,disc_jockey,0
4,train_corrupt-3701,paul_melchers,nationality,united_states,0
